In [27]:
%%time
import os
import json

import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.utils import unique_name_from_base

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = get_execution_role()
prefix = 'wenet'
output_path = f"s3://{bucket}/{prefix}"

print("boto3.__version__:{}".format(boto3.__version__))
print("sagemaker.__version__:{}".format(sagemaker.__version__))
print("bucket:{}".format(bucket))
print("role:{}".format(role))

boto3.__version__:1.26.8
sagemaker.__version__:2.116.0
bucket:sagemaker-us-east-1-348052051973
role:arn:aws:iam::348052051973:role/service-role/AmazonSageMakerServiceCatalogProductsExecutionRole
CPU times: user 295 ms, sys: 20.2 ms, total: 315 ms
Wall time: 445 ms


In [28]:
%%markdown
Copy the wenet/examples/librispeech/s0/*.sh and wenet/examples/librispeech/s0/local to wenet/ as requested by Sagemaker
Overwrite the wenet/wenet/bin/train.py with the given one
Change the /root/wenet to /opt/ml/input in all data.list files (especially for train_960 and dev)
The "Librispeech" in data.list file in Github has the wrong captalization because it's wrong when I upload it. Please change it yourself!

Copy the wenet/examples/librispeech/s0/*.sh and wenet/examples/librispeech/s0/local to wenet/ as requested by Sagemaker
Overwrite the wenet/wenet/bin/train.py with the given one
Change the /root/wenet to /opt/ml/input in all data.list files (especially for train_960 and dev)
The "Librispeech" in data.list file in Github has the wrong captalization because it's wrong when I upload it. Please change it yourself!


In [49]:
instance_type = "ml.p3.16xlarge"
max_run = 432000
checkpoint_s3_uri = f"s3://{bucket}/{prefix}/checkpoints"

hyperparameters = {
    'datadir':'/opt/ml/input/data/training',
    'stage': '4',
    'stop_stage': '4',
    'train_config': 'examples/librispeech/s0/conf/train_conformer.yaml',
    'model_dir': '/opt/ml/model',
    'checkpoint_dir': '/opt/ml/checkpoints',
    'output_dir': '/opt/ml/output/data',
}

est = PyTorch(
    entry_point="run-8gpu.sh",
    source_dir="./wenet",
    framework_version="1.11.0",
    py_version="py38",
    role=role,
    instance_count=1,
    instance_type=instance_type,
    volume_size=200,
    disable_profiler=True,
    debugger_hook_config=False,
    base_job_name=prefix,
    hyperparameters = hyperparameters,
    checkpoint_s3_uri = checkpoint_s3_uri,
    output_path = f"s3://{bucket}/{prefix}/",
    # keep_alive_period_in_seconds=1800,
    max_run = max_run,
    tags = [{"Key": "team", "Value": "asr"}, {"Key": "person", "Value": "andrew"}, {"Key": "project", "Value": "abc"}],
)

In [50]:
from sagemaker.inputs import TrainingInput
prefix_dataset = "wenet/export"
loc =f"s3://{bucket}/{prefix_dataset}"

training = TrainingInput(
    s3_data_type='S3Prefix', # Available Options: S3Prefix | ManifestFile | AugmentedManifestFile
    s3_data=loc,
    distribution='FullyReplicated', # Available Options: FullyReplicated | ShardedByS3Key 
    input_mode='FastFile'
)

In [ ]:
%%time
job_name = est.fit({"training":training})
#job_name = est.fit()

2022-12-01 07:41:57 Starting - Starting the training job.........
2022-12-01 07:43:04 Starting - Preparing the instances for training......
2022-12-01 07:44:22 Downloading - Downloading input data......
2022-12-01 07:45:07 Training - Downloading the training image.......................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-12-01 07:49:06,975 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-12-01 07:49:07,053 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-12-01 07:49:07,060 sagemaker_pytorch_container.training INFO     Invoking user training script.

2022-12-01 07:49:03 Training - Training image download completed. Training in progress.2022-12-01 07:49:20,067 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "trainin

In [52]:
model_data = est.model_data
print("Model artifact saved at:\n", model_data)

Model artifact saved at:
 s3://sagemaker-us-east-1-348052051973/wenet/wenet-2022-12-01-06-28-23-628/output/model.tar.gz
